In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import scale
from sklearn import model_selection  # cross_validation
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error
import math as mt

C:\Users\Andrew\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data = pd.read_csv('C:/Users/Andrew/Documents/dat11syd/lessons/lesson-07/hwk/seattleWeather_1948-2017.csv')
data = data[pd.notnull(data['RAIN'])]
data = data[pd.notnull(data['PRCP'])]
DATE = data['DATE']
del data['DATE']
data['Tdif'] = (data['TMAX'] - data['TMIN'])
del data['TMAX']
del data['TMIN']
data = data.drop(data.index[1419])
data.RAIN = data.RAIN.astype(int)
#data=(data-data.mean())/data.std()
data['DATE'] = DATE
pbefore = []
j = -1
for i in data['DATE']:
    j = j + 1
    if j in (0, 1, 2):
        pbefore.append(np.nan)
    else:
        pbefore.append((data.iloc[j - 1]['PRCP'] + data.iloc[j - 2]['PRCP'] + data.iloc[j - 3]['PRCP'])/3)
data['pbefore'] = pbefore
data = data[pd.notnull(data['pbefore'])]

In [3]:
data['DATE'] =  pd.to_datetime(data['DATE'])
data.head()

,PRCP,RAIN,Tdif,DATE,pbefore
3,0.31,1,11,1948-01-04,0.493333
4,0.17,1,13,1948-01-05,0.440000
5,0.44,1,9,1948-01-06,0.300000
6,0.41,1,10,1948-01-07,0.306667
7,0.04,1,13,1948-01-08,0.340000


In [4]:
split_date = pd.datetime(2009,12,31)

df = data.loc[data['DATE'] <= split_date]
df_val = data.loc[data['DATE'] > split_date]
df_val.head()

,PRCP,RAIN,Tdif,DATE,pbefore
22646,0.40,1,7,2010-01-01,0.116667
22647,0.06,1,4,2010-01-02,0.230000
22648,0.03,1,9,2010-01-03,0.243333
22649,0.98,1,5,2010-01-04,0.163333
22650,0.14,1,6,2010-01-05,0.356667


In [5]:
print(df.shape)
print(df_val.shape)

(22639, 5)
(2905, 5)


In [6]:
df.corr()

,PRCP,RAIN,Tdif,pbefore
PRCP,1.000000,0.514964,-0.339321,0.273676
RAIN,0.514964,1.000000,-0.542323,0.292677
Tdif,-0.339321,-0.542323,1.000000,-0.325158
pbefore,0.273676,0.292677,-0.325158,1.000000


In [7]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

coef1 = []
coef2 = []
coef3 = []

for i in range(0,100):
    feature_cols = ['pbefore', 'Tdif']
    X = df[feature_cols]
    y = df['RAIN']    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=i)
    reg = linear_model.LogisticRegression()
    reg.fit(X_train, y_train)    
    coef1.append(reg.coef_[0][0])
    coef2.append(reg.coef_[0][1])
    coef3.append(reg.intercept_)
    print(reg.coef_)
    
print(np.average(coef1))
print(np.average(coef2))
print(np.average(coef3))

a = np.average(coef1)
b = np.average(coef2)
c = np.average(coef3)

[[ 2.08824346 -0.23334299]]
[[ 1.8697643  -0.23634386]]
[[ 2.09708314 -0.23220818]]
[[ 1.98404271 -0.23423873]]
[[ 1.98416155 -0.22936392]]
[[ 2.04921426 -0.22815082]]
[[ 1.97138731 -0.23115302]]
[[ 2.11866351 -0.22922391]]
[[ 2.03783878 -0.23069324]]
[[ 2.14874742 -0.2289298 ]]
[[ 2.07329934 -0.22929482]]
[[ 2.07594294 -0.22786179]]
[[ 1.98518529 -0.23435215]]
[[ 2.05039583 -0.23367948]]
[[ 2.00632169 -0.23021387]]
[[ 2.06805046 -0.22924944]]
[[ 2.03312036 -0.23051562]]
[[ 2.04049835 -0.23067773]]
[[ 2.20055716 -0.23130778]]
[[ 2.12932189 -0.23003766]]
[[ 2.02256247 -0.23347736]]
[[ 2.11258649 -0.22876421]]
[[ 2.0675246  -0.23243109]]
[[ 2.08483981 -0.23334979]]
[[ 2.00825027 -0.23073457]]
[[ 1.96832276 -0.23125468]]
[[ 1.99561219 -0.23089198]]
[[ 2.09294109 -0.23535394]]
[[ 2.00911463 -0.23209627]]
[[ 2.36922099 -0.23358104]]
[[ 2.08273872 -0.23170937]]
[[ 1.99081561 -0.23237238]]
[[ 2.15551532 -0.23228334]]
[[ 2.00144656 -0.23304519]]
[[ 2.07486465 -0.23172604]]
[[ 1.99687358 -0.231

In [8]:
Y = data.RAIN.unique()
j = -1
pred = []
for i in df_val['DATE']:
    j = j + 1
    y_pred = 1/(1+mt.exp(-(a * df_val.iloc[j]['pbefore'] + b * df_val.iloc[j]['Tdif'] + c)))
    if y_pred >= 0.5:
        y_pred = True
    else:
        y_pred = False
    pred.append(y_pred)    

In [9]:
check = pd.read_csv('C:/Users/Andrew/Documents/dat11syd/lessons/lesson-07/hwk/seattleWeather_1948-2017.csv')
check = check[pd.notnull(check['RAIN'])]
check = check[pd.notnull(check['PRCP'])]
check['DATE'] =  pd.to_datetime(check['DATE'])
split_date = pd.datetime(2009,12,31)
check_val = check.loc[check['DATE'] > split_date]
check_val.head()

,DATE,PRCP,TMAX,TMIN,RAIN
22646,2010-01-01,0.40,52,45,True
22647,2010-01-02,0.06,47,43,True
22648,2010-01-03,0.03,48,39,True
22649,2010-01-04,0.98,49,44,True
22650,2010-01-05,0.14,50,44,True


In [10]:
correct = []
j = -1
for i in check_val['RAIN']:
    j = j + 1
    if pred[j] == i:
        correct.append(1)
    else:
        correct.append(0)
print('Percentage Estimated Correctly: ' + str(np.sum(correct)/len(correct)*100) + '%')  

Percentage Estimated Correctly: 77.0051635111876%


In [11]:
check_val.head()

,DATE,PRCP,TMAX,TMIN,RAIN
22646,2010-01-01,0.40,52,45,True
22647,2010-01-02,0.06,47,43,True
22648,2010-01-03,0.03,48,39,True
22649,2010-01-04,0.98,49,44,True
22650,2010-01-05,0.14,50,44,True


In [12]:
check_val.RAIN = check_val.RAIN.astype(int)
check_val.head()

C:\Users\Andrew\Anaconda3\lib\site-packages\pandas\core\generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,DATE,PRCP,TMAX,TMIN,RAIN
22646,2010-01-01,0.40,52,45,1
22647,2010-01-02,0.06,47,43,1
22648,2010-01-03,0.03,48,39,1
22649,2010-01-04,0.98,49,44,1
22650,2010-01-05,0.14,50,44,1


In [33]:
print('Null Accuracy is ' + str(1 - np.sum(check_val['RAIN'])/len(check_val['RAIN'])))

Null Accuracy is 0.5524956970740103


In [27]:
conf = []
j = - 1
for i in check_val['RAIN']:
    j = j + 1
    if pred[j] == True and i == 1:
        conf.append('TP')
    if pred[j] == True and i == 0:
        conf.append('FP')
    if pred[j] == False and i == 0:
        conf.append('TN')
    else:
        conf.append('FN')
        

In [29]:
print('Number of TP is ' + str(sum(1 for i in conf if i == 'TP')))
print('Number of FP is ' + str(sum(1 for i in conf if i == 'FP')))
print('Number of TN is ' + str(sum(1 for i in conf if i == 'TN')))
print('Number of FN is ' + str(sum(1 for i in conf if i == 'FN')))


Number of TP is 961
Number of FP is 329
Number of TN is 1276
Number of FN is 1629
